In [1]:
import pandas as pd
import numpy as np
import csv as csv
import math
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from sklearn import cross_validation
from sklearn import metrics
from sklearn.cross_validation import train_test_split,cross_val_score
import time
import time
from datetime import date
import datetime

%matplotlib inline

#### Concatenate the two training DataFrames

In [2]:
train_df1 = pd.read_csv('../hackerrank-predict-email-opens-dataset/training_dataset.csv/training_dataset.csv', header=0)  
train_df2 = pd.read_csv('../hackerrank-predict-email-opens-dataset/training_dataset.csv/training_dataset_complement.csv', header=0)  
train_df = pd.concat([train_df1,train_df2])
train_df.shape

(486048, 54)

#### Dropping few columns

In [3]:
train_df = train_df.drop(['user_id','mail_id','mail_type','clicked','hacker_timezone',
                          'mail_category','sent_time','unsubscribed',
                          'open_time','click_time','unsubscribe_time','hacker_created_at'], axis=1) 
train_df.head()

,last_online,contest_login_count,contest_login_count_1_days,contest_login_count_30_days,contest_login_count_365_days,contest_login_count_7_days,contest_participation_count,contest_participation_count_1_days,contest_participation_count_30_days,contest_participation_count_365_days,...,submissions_count_contest,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days
0,1.459520e+09,1,0,0,1,0,1,0,0,1,...,0,0,0,0,0,13,0,0,13,0
1,1.461210e+09,3,0,1,3,0,3,0,1,3,...,16,0,3,16,0,83,0,43,83,4
2,1.463411e+09,3,0,0,3,0,3,0,0,3,...,0,0,0,0,0,16,0,3,16,0
3,1.462768e+09,3,0,0,3,0,3,0,0,3,...,16,0,0,16,0,85,0,9,85,0
4,1.461248e+09,5,0,0,5,0,13,0,0,13,...,17,0,1,17,0,43,0,0,43,0


#### Converting Boolean to Int

In [4]:
train_df['opened'] = train_df.opened.map({True:1,False:0}).astype(int)
train_df['hacker_confirmation'] = train_df.hacker_confirmation.map({True:1,False:0}).astype(int)
train_df.shape


(486048, 42)

#### Dropping rows with NA or Null Values (last_online is the only attribute with NA or Null values)

In [5]:
train_df = train_df.dropna(subset=['last_online'],axis=0)
train_df.shape

(485471, 42)

#### Sorting Columns to data constancy

In [6]:
train_df = train_df.reindex_axis(sorted(train_df.columns), axis=1)
train_df.shape

(485471, 42)

#### Adding opened attribute to the first column of DataFrame

In [7]:
train_df = train_df.reindex_axis(['opened'] + list([col for col in train_df.columns if col != 'opened']), axis=1)
train_df.head()

,opened,contest_login_count,contest_login_count_1_days,contest_login_count_30_days,contest_login_count_365_days,contest_login_count_7_days,contest_participation_count,contest_participation_count_1_days,contest_participation_count_30_days,contest_participation_count_365_days,...,submissions_count_contest,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days
0,1,1,0,0,1,0,1,0,0,1,...,0,0,0,0,0,13,0,0,13,0
1,0,3,0,1,3,0,3,0,1,3,...,16,0,3,16,0,83,0,43,83,4
2,0,3,0,0,3,0,3,0,0,3,...,0,0,0,0,0,16,0,3,16,0
3,0,3,0,0,3,0,3,0,0,3,...,16,0,0,16,0,85,0,9,85,0
4,0,5,0,0,5,0,13,0,0,13,...,17,0,1,17,0,43,0,0,43,0


#### Check for NA values

In [8]:
hasany = False
for cl in train_df.columns.values:
    hasNa = train_df[cl].isnull().sum() > 0
    if hasNa:
        hasany = True
        print cl,train_df[cl].isnull().sum()
if not hasany:
    print 'No NA found'
    

No NA found


#### Check data types for your model

In [9]:
for cl in train_df.columns.values:
    print cl,train_df[cl].dtype

opened int32
contest_login_count int64
contest_login_count_1_days int64
contest_login_count_30_days int64
contest_login_count_365_days int64
contest_login_count_7_days int64
contest_participation_count int64
contest_participation_count_1_days int64
contest_participation_count_30_days int64
contest_participation_count_365_days int64
contest_participation_count_7_days int64
forum_comments_count int64
forum_count int64
forum_expert_count int64
forum_questions_count int64
hacker_confirmation int32
ipn_count int64
ipn_count_1_days int64
ipn_count_30_days int64
ipn_count_365_days int64
ipn_count_7_days int64
ipn_read int64
ipn_read_1_days int64
ipn_read_30_days int64
ipn_read_365_days int64
ipn_read_7_days int64
last_online float64
submissions_count int64
submissions_count_1_days int64
submissions_count_30_days int64
submissions_count_365_days int64
submissions_count_7_days int64
submissions_count_contest int64
submissions_count_contest_1_days int64
submissions_count_contest_30_days int64
su

In [10]:
from sklearn import linear_model
from sklearn import tree
from sklearn import svm
from sklearn import ensemble 
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import metrics

train_data = train_df.values
data = train_data[0::,1::]
target =  train_data[0::,0]
x_train, x_test, y_train, y_test = train_test_split(train_data[0::,1::], train_data[0::,0], 
                            test_size = 0.2, random_state = 123) # Split training/test.

# hipotese = linear_model.LogisticRegression(C=1e5)
hipotese = tree.DecisionTreeClassifier(random_state=1234)
# hipotese = ensemble.RandomForestClassifier(random_state=1234)

# hipotese = svm.SVC()



In [18]:
from sklearn.decomposition import PCA, IncrementalPCA

n_components = 2

reduction_model = PCA(n_components=n_components)

reduction_model = reduction_model.fit(x_train)
x_train_transformed = reduction_model.transform(x_train)


hipotese.fit(x_train_transformed, y_train)
x_test_transformed = reduction_model.transform(x_test)

y_true, y_pred = y_test, hipotese.predict(x_test_transformed) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit


             precision    recall  f1-score   support

        0.0       0.79      0.83      0.81     64915
        1.0       0.61      0.54      0.58     32180

avg / total       0.73      0.74      0.73     97095



In [19]:
from sklearn.decomposition import PCA, IncrementalPCA


n_components = 2
reduction_model = IncrementalPCA(n_components=n_components, batch_size=10)


reduction_model = reduction_model.fit(x_train)
x_train_transformed = reduction_model.transform(x_train)
hipotese.fit(x_train_transformed, y_train)
x_test_transformed = reduction_model.transform(x_test)

y_true, y_pred = y_test, hipotese.predict(x_test_transformed) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit


             precision    recall  f1-score   support

        0.0       0.79      0.83      0.81     64915
        1.0       0.61      0.54      0.58     32180

avg / total       0.73      0.74      0.73     97095



In [20]:
from sklearn import decomposition
from sklearn.cluster import MiniBatchKMeans

n_components = 2
rng = 123456
options = [('Eigenfaces - RandomizedPCA',
     decomposition.RandomizedPCA(n_components=n_components, whiten=True),
     True),

    ('Non-negative components - NMF',
     decomposition.NMF(n_components=n_components, init='nndsvda', tol=5e-3),
     False),

    ('Independent components - FastICA',
     decomposition.FastICA(n_components=n_components, whiten=True),
     True),

    ('Sparse comp. - MiniBatchSparsePCA',
     decomposition.MiniBatchSparsePCA(n_components=n_components, alpha=0.8,
                                      n_iter=100, batch_size=3,
                                      random_state=rng),
     True),

    ('MiniBatchDictionaryLearning',
        decomposition.MiniBatchDictionaryLearning(n_components=15, alpha=0.1,
                                                  n_iter=50, batch_size=3,
                                                  random_state=rng),
     True),

    ('Cluster centers - MiniBatchKMeans',
        MiniBatchKMeans(n_clusters=n_components, tol=1e-3, batch_size=20,
                        max_iter=50, random_state=rng),
     True),

    ('Factor Analysis components - FA',
     decomposition.FactorAnalysis(n_components=n_components, max_iter=2),
     True)]

for name,x_train_transformed,boolean in options:
    print(name)
    
    reduction_model = reduction_model.fit(x_train)
    x_train_transformed = reduction_model.transform(x_train)

    hipotese.fit(x_train_transformed, y_train)
    x_test_transformed = reduction_model.transform(x_test)

    y_true, y_pred = y_test, hipotese.predict(x_test_transformed) # Get our predictions
    print(classification_report(y_true, y_pred)) # Classification on each digit


Eigenfaces - RandomizedPCA
             precision    recall  f1-score   support

        0.0       0.79      0.83      0.81     64915
        1.0       0.61      0.54      0.58     32180

avg / total       0.73      0.74      0.73     97095

Non-negative components - NMF
             precision    recall  f1-score   support

        0.0       0.79      0.83      0.81     64915
        1.0       0.61      0.54      0.58     32180

avg / total       0.73      0.74      0.73     97095

Independent components - FastICA
             precision    recall  f1-score   support

        0.0       0.79      0.83      0.81     64915
        1.0       0.61      0.54      0.58     32180

avg / total       0.73      0.74      0.73     97095

Sparse comp. - MiniBatchSparsePCA
             precision    recall  f1-score   support

        0.0       0.79      0.83      0.81     64915
        1.0       0.61      0.54      0.58     32180

avg / total       0.73      0.74      0.73     97095

MiniBatchDiction

In [14]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np
n_components=2
reduction_model = TruncatedSVD(n_components)


reduction_model = reduction_model.fit(x_train)
x_train_transformed = reduction_model.transform(x_train)

hipotese.fit(x_train_transformed, y_train)
x_test_transformed = reduction_model.transform(x_test)

y_true, y_pred = y_test, hipotese.predict(x_test_transformed) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit


    

             precision    recall  f1-score   support

        0.0       0.78      0.83      0.81     64915
        1.0       0.61      0.54      0.57     32180

avg / total       0.73      0.73      0.73     97095



In [ ]:
from sklearn.decomposition import PCA, KernelPCA

reduction_model = KernelPCA(n_components=2,kernel="rbf", gamma=10)

reduction_model = reduction_model.fit(x_train)
x_train_transformed = reduction_model.transform(x_train)

hipotese.fit(x_train_transformed, y_train)
x_test_transformed = reduction_model.transform(x_test)

y_true, y_pred = y_test, hipotese.predict(x_test_transformed) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit




In [13]:
from sklearn.decomposition import MiniBatchDictionaryLearning
reduction_model = MiniBatchDictionaryLearning(n_components=30, alpha=1, n_iter=20)

V = reduction_model.fit(x_train).components_



reduction_model = reduction_model.fit(x_train)
x_train_transformed = reduction_model.transform(x_train)

hipotese.fit(x_train_transformed, y_train)
x_test_transformed = reduction_model.transform(x_test)

y_true, y_pred = y_test, hipotese.predict(x_test_transformed) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit


    


C:\Users\alvaro.joao.silvino\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\linear_model\least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 25 iterations, i.e. alpha=2.619e-02, with an active set of 9 regressors, and the smallest cholesky pivot element being 4.942e-08
  ConvergenceWarning)
C:\Users\alvaro.joao.silvino\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\linear_model\least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 8 iterations, i.e. alpha=1.496e-01, with an active set of 8 regressors, and the smallest cholesky pivot element being 8.689e-08
  ConvergenceWarning)
C:\Users\alvaro.joao.silvino\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\linear_model\least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 9 iterations, alpha=1.495e

             precision    recall  f1-score   support

        0.0       0.76      0.80      0.78     64915
        1.0       0.55      0.49      0.52     32180

avg / total       0.69      0.70      0.69     97095



In [12]:
from sklearn.decomposition import FastICA
# Compute ICA
reduction_model = FastICA(n_components=3)

reduction_model = reduction_model.fit(x_train)
A_ = reduction_model.mixing_  # Get estimated mixing matrix
x_train_transformed = reduction_model.transform(x_train)

hipotese.fit(x_train_transformed, y_train)
x_test_transformed = reduction_model.transform(x_test)

y_true, y_pred = y_test, hipotese.predict(x_test_transformed) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit




             precision    recall  f1-score   support

        0.0       0.79      0.83      0.81     64915
        1.0       0.62      0.54      0.58     32180

avg / total       0.73      0.74      0.73     97095



In [14]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
reduction_model = NMF(n_components=2, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)

reduction_model = reduction_model.fit(x_train)
A_ = reduction_model.mixing_  # Get estimated mixing matrix
x_train_transformed = reduction_model.transform(x_train)

hipotese.fit(x_train_transformed, y_train)
x_test_transformed = reduction_model.transform(x_test)

y_true, y_pred = y_test, hipotese.predict(x_test_transformed) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit





NameError: name 'tfidf' is not defined

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

reduction_model = LatentDirichletAllocation(n_topics=3, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)

reduction_model = reduction_model.fit(x_train)
A_ = reduction_model.mixing_  # Get estimated mixing matrix
x_train_transformed = reduction_model.transform(x_train)

hipotese.fit(x_train_transformed, y_train)
x_test_transformed = reduction_model.transform(x_test)

y_true, y_pred = y_test, hipotese.predict(x_test_transformed) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit




